In [3]:
import re
import os

import pandas as pd

In [4]:
course_code_to_faculties = {
    'GC':'Asian School of the Environment', 'HW':'Language and Communication Centre', 'HY':'School of Humanities', 
    'LS':'Centre for Modern Languages', 'SP':'University Scholars Programme', 'AB':'Nanyang Business School', 
    'AC':'Nanyang Business School', 'BE':'Nanyang Business School', 'CC':'Interdisciplinary Collaborative Core', 
    'HE':'School of Social Sciences', 'ML':'Interdisciplinary Collaborative Core', 'ET':'NTUpreneur', 
    'DD':'School Of Art, Design and Media', 'DN':'School Of Art, Design and Media', 'HS':'School of Social Sciences', 
    'DF':'School Of Art, Design and Media', 'DP':'School Of Art, Design and Media', 'DT':'School Of Art, Design and Media', 
    'DV':'School Of Art, Design and Media', 'HR':'School of Humanities', 'CB':'School of Chemistry, Chemical Engineering and Biotechnology', 
    'CZ':'School of Computer Science and Engineering', 'EE':'School of Electrical and Electronic Engineering', 
    'EG':'College of Engineering', 'MA':'School of Mechanical and Aerospace Engineering', 
    'MH':'School of Physical and Mathematical Sciences', 'MS':'School of Materials Science and Engineering', 
    'PH':'School of Physical and Mathematical Sciences', 'BG':'School of Chemistry, Chemical Engineering and Biotechnology', 
    'BS':'School of Biological Sciences', 'CH':'School of Chemistry, Chemical Engineering and Biotechnology', 
    'ES':'Asian School of the Environment', 'AD':'Nanyang Business School', 'CE':'School of Computer Science and Engineering', 
    'CV':'School of Civil and Environmental Engineering', 'EN':'School of Civil and Environmental Engineering', 
    'CM':'School of Chemistry, Chemical Engineering and Biotechnology', 'PS':'School of Physical and Mathematical Sciences', 
    'HC':'School of Humanities', 'HG':'School of Humanities', 'HL':'School of Humanities', 'HP':'School of Social Sciences', 
    'CS':'Wee Kim Wee School of Communication and Information', 'FL':'Wee Kim Wee School of Communication and Information', 
    'HA':'School of Social Sciences', 'MT':'School of Civil and Environmental Engineering', 'DM':'School Of Art, Design and Media',
    'DR':'School Of Art, Design and Media', 'BA':'Nanyang Business School', 'BH':'Nanyang Business School', 'BL':'Nanyang Business School', 
    'BF':'Nanyang Business School', 'BC':'Nanyang Business School', 'BT':'Nanyang Business School', 'BR':'Nanyang Business School', 
    'BU':'Nanyang Business School', 'BM':'Nanyang Business School',
    'SC':'School of Computer Science and Engineering', 'IE':'School of Electrical and Electronic Engineering', 'DA':'School Of Art, Design and Media', 
    'HH':'School of Humanities', 'FE':'College of Engineering', 'IM':'School of Electrical and Electronic Engineering', 
    'RE':'Renaissance Engineering Programme', 'HF':'School of Humanities', 'HZ':'School of Humanities', 'MD':'Lee Kong Chian School of Medicine',
    'EM':'School of Civil and Environmental Engineering', 'HT':'School of Humanities', 'HQ':'School of Humanities', 
    'HU':'School of Social Sciences', 'LA':'Centre for Modern Languages', 'LF':'Centre for Modern Languages', 
    'LG':'Centre for Modern Languages', 'LJ':'Centre for Modern Languages', 'LK':'Centre for Modern Languages',
    'LM':'Centre for Modern Languages', 'LR':'Centre for Modern Languages', 'LT':'Centre for Modern Languages', 
    'LI':'Centre for Modern Languages', 'LL':'Centre for Modern Languages', 'LV':'Centre for Modern Languages', 
    'LX':'Centre for Modern Languages', 'LH':'Centre for Modern Languages', 'SS':'National Institute of Education', 
    'SU':'School of Civil and Environmental Engineering', 'CY':'CN Yang Scholars Programme', 
    'HN':'School of Social Sciences', 'HJ':'School of Social Sciences'
}

In [1]:
course_code_to_disciplines = {
    'GC':'Environment Earth Systems', 'HW':'Language and Communication', 'HY':'Philosophy', 
    'LS':'Languages', 'SP':'University Scholars Programme', 'AB':'Business/Accountancy', 'AC':'Business/Accountancy', 
    'BE':'Business/Accountancy', 'CC':'Interdisciplinary Collaborative Core', 'HE':'Economics',
    'ML':'Interdisciplinary Collaborative Core', 'ET':'Entrepreneurship', 'DD':'Interdisciplinary ', 'DN':'Art, Design and Media', 
    'HS':'Sociology', 'DF':'Art, Design and Media', 'DP':'Art, Design and Media', 'DT':'Art, Design and Media', 'DV':'Art, Design and Media',
    'HR':'History', 'CB':'Chemistry, Chemical Engineering and Biotechnology', 'CZ':'Computing', 'EE':'Electrical Engineering', 
    'EG':'Engineering', 'MA':'Mechanical/Aerospace Engineering', 'MH':'Mathematics', 'MS':'Materials Science', 'PH':'Physics',
    'BG':'Chemistry, Chemical Engineering and Biotechnology', 'BS':'Biological Science', 'CH':'Chemistry, Chemical Engineering and Biotechnology', 
    'ES':'Environment Earth Systems', 'AD':'Business/Accountancy', 'CE':'Computing', 'CV':'Civil/Environmental Engineering', 
    'EN':'Civil/Environmental Engineering', 'CM':'Chemistry, Chemical Engineering and Biotechnology',
    'PS':'Physical and Mathematical Sciences', 'HC':'Chinese', 'HG':'Linguistics', 'HL':'English Literature', 'HP':'Psychology', 
    'CS':'Communication Studies', 'FL':'Communication Studies', 'HA':'Public Policy and Global Affairs', 'MT':'Maritime Studies', 'DM':'Art, Design and Media',
    'DR':'Art, Design and Media', 'BA':'Business/Accountancy', 'BH':'Business/Accountancy', 'BL':'Business/Accountancy', 
    'BF':'Business/Accountancy', 'BC':'Business/Accountancy', 'BT':'Business/Accountancy', 'BR':'Business/Accountancy', 
    'BU':'Business/Accountancy', 'BM':'Business/Accountancy', 'SC':'Computing', 'IE':'Electrical Engineering', 
    'DA':'Art, Design and Media', 'HH':'History', 'FE':'Engineering', 'IM':'Electrical Engineering', 'RE':'Renaissance Engineering', 
    'HF':'Chinese', 'HZ':'English Literature', 'MD':'Medicine', 'EM':'Civil/Environmental Engineering', 'HT':'Chinese', 
    'HQ':'English Literature', 'HU':'Geography', 'LA':'Languages', 'LF':'Languages', 'LG':'Languages', 'LJ':'Languages', 
    'LK':'Languages', 'LM':'Languages', 'LR':'Languages', 'LT':'Languages', 'LI':'Languages', 'LL':'Languages', 
    'LV':'Languages', 'LX':'Languages', 'LH':'Languages', 'SS':'Sport Science & Management', 
    'SU':'Civil/Environmental Engineering', 'CY':'CN Yang Scholars Programme', 'HN':'Social Science', 'HJ':'Social Science'
}

In [6]:
def parse_prerequisites(prerequisites):
    match_for_course_code_pattern_1 = re.search(r'\b[A-Z]{2}\d{4}\b', prerequisites)
    match_for_course_code_pattern_2 = re.search(r'\b[A-Z]{3}\d{2}[A-Z]{1}\b', prerequisites)
    
    if prerequisites != '' and (match_for_course_code_pattern_1 or match_for_course_code_pattern_2):
        prereqs_split = re.split(r'(?<![a-zA-Z])OR(?![a-zA-Z])|(?<![a-zA-Z])or(?![a-zA-Z])', prerequisites)

        list_of_prereqs = []
    
        for elem in prereqs_split:
            elem_split = elem.split('&')
            
            elem_split = [x.strip() for x in elem_split]
            
            new_list = []
            
            for x in elem_split:
                if (x.find('Corequisite') != -1):
                    return
                elif (re.search(r'\b[A-Z]{2}\d{4}\b', x)):
                    results = re.findall(r'\b[A-Z]{2}\d{4}\b', x)
                    new_list.append(results[0])                    
                elif (re.search(r'\b[A-Z]{3}\d{2}[A-Z]{1}\b', x)):
                    results = re.findall(r'\b[A-Z]{3}\d{2}[A-Z]{1}\b', x)
                    new_list.append(results[0])
                elif (re.search(r'\b[A-Z]{3}\d{3}\b', x)):
                    results = re.findall(r'\b[A-Z]{3}\d{3}\b', x)
                    new_list.append(results[0])
            
            if (len(new_list) > 0):
                list_of_prereqs.append(new_list)
    
        return list_of_prereqs

In [15]:
def assign_faculty(course_code_to_faculties, course_code):
    match_for_course_code_pattern_1 = re.search(r'\b[A-Z]{2}\d{4}\b', course_code)
    match_for_course_code_pattern_2 = re.search(r'\b[A-Z]{3}\d{2}[A-Z]{1}\b', course_code)
    
    try:
        if (match_for_course_code_pattern_1):
            return course_code_to_faculties[course_code[0:2]]

        if (match_for_course_code_pattern_2):
            return 'National Institute of Education'
    except Exception:
        print(course_code)

In [16]:
def assign_discipline(course_code_to_disciplines, course_code):
    match_for_course_code_pattern_1 = re.search(r'\b[A-Z]{2}\d{4}\b', course_code)
    match_for_course_code_pattern_2 = re.search(r'\b[A-Z]{3}\d{2}[A-Z]{1}\b', course_code)
    
    try:
        if (match_for_course_code_pattern_1):
            return course_code_to_disciplines[course_code[0:2]]

        if (match_for_course_code_pattern_2):
            return 'National Institute of Education'
    except Exception:
        print(course_code)

In [17]:
def parse_mutually_exclusives(mutually_exclusives):
    if mutually_exclusives == '':
        return

    list_of_mutually_exclusives = []
    
    split = mutually_exclusives.split(',');
    
    for x in split:
        x.strip()
        list_of_mutually_exclusives.append(x)
        
    return list_of_mutually_exclusives

In [22]:
def process_data(modules_df):  
    modules_df.fillna('', inplace=True)
    modules_df.drop_duplicates(subset=['Course Code'], inplace=True)
    modules_df.replace(regex=r'([0-9]{0,1}.[0-9]{1})\sAU', value=r'\1', inplace=True)
    
    modules_df['Academic Units'] = pd.to_numeric(modules_df['Academic Units'])
    modules_df['Academic Units'] = modules_df['Academic Units'].fillna(0)
    modules_df = modules_df.astype({'Academic Units' : 'int64'})
    modules_df.reset_index(inplace=True, drop=True)
    
    modules_df['Prerequisites'] = modules_df['Prerequisites'].map(lambda x: parse_prerequisites(x))
    modules_df['Faculty'] = modules_df['Course Code'].map(lambda x: assign_faculty(course_code_to_faculties, x))
    modules_df['Discipline'] = modules_df['Course Code'].map(lambda x: assign_discipline(course_code_to_disciplines, x))
    modules_df['Mutually Exclusive'] = modules_df['Mutually Exclusive'].map(lambda x: parse_mutually_exclusives(x))
    
    return modules_df

In [23]:
def merge_all_modules(modules1, modules2, *argv):
    list_of_modules_to_merge = [modules1, modules2]
    
    for m in argv:
        list_of_modules_to_merge.append(m)
        
    merged_modules = pd.concat(list_of_modules_to_merge)
    merged_modules.drop_duplicates(subset=['Course Code'], inplace=True)
    merged_modules = merged_modules[merged_modules['Course Code'] != '']
    merged_modules.reset_index(inplace=True, drop=True)
    
    return merged_modules

In [24]:
sem1_single_degree_modules = pd.read_csv('sem_1_single_degree_modules.csv', index_col=0)
sem1_double_degree_modules = pd.read_csv('sem_1_double_degree_modules.csv', index_col=0)
sem1_rest_of_the_modules = pd.read_csv('sem_1_rest_of_the_modules.csv', index_col=0)

In [25]:
sem1_single_degree_modules_processed = process_data(sem1_single_degree_modules)
sem1_double_degree_modules_processed = process_data(sem1_double_degree_modules)
sem1_rest_of_the_modules_processed = process_data(sem1_rest_of_the_modules)

In [26]:
sem2_single_degree_modules = pd.read_csv('sem_2_single_degree_modules.csv', index_col=0)
sem2_double_degree_modules = pd.read_csv('sem_2_double_degree_modules.csv', index_col=0)
sem2_rest_of_the_modules = pd.read_csv('sem_2_rest_of_the_modules.csv', index_col=0)

In [27]:
sem2_single_degree_modules_processed = process_data(sem2_single_degree_modules)
sem2_double_degree_modules_processed = process_data(sem2_double_degree_modules)
sem2_rest_of_the_modules_processed = process_data(sem2_rest_of_the_modules)

In [28]:
sem1_merged_modules = merge_all_modules(sem1_single_degree_modules_processed, sem1_double_degree_modules_processed, sem1_rest_of_the_modules_processed)

In [29]:
sem2_merged_modules = merge_all_modules(sem2_single_degree_modules_processed, sem2_double_degree_modules_processed, sem2_rest_of_the_modules_processed)

In [30]:
all_merged_modules = merge_all_modules(sem1_merged_modules, sem2_merged_modules)

In [32]:
all_merged_modules.to_csv(os.path.join(os.getcwd(), "all_modules.csv"), index=False)

In [33]:
all_merged_modules.describe(include='all')

,Course Code,Course Name,Academic Units,Faculty,BDE,Grade Type,Prerequisites,Mutually Exclusive,Course Information,Discipline
count,1872,1872,1872.000000,1872,1872,1872,946,742,1872,1872
unique,1872,1686,NaN,24,2,2,571,719,1724,37
top,GC0001,INTRODUCTION TO DATA SCIENCE & ARTIFICIAL INTE...,NaN,School of Humanities,Yes,Letter Graded,[[HL1001]],"[HG5012, HG8012]",,Business/Accountancy
freq,1,9,NaN,279,1592,1838,44,2,5,182
mean,NaN,NaN,3.146902,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,0.650865,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
all_merged_modules.head(10)

,Course Code,Course Name,Academic Units,Faculty,BDE,Grade Type,Prerequisites,Mutually Exclusive,Course Information,Discipline
0,GC0001,SUSTAINABILITY: SEEING THROUGH THE HAZE,1,Asian School of the Environment,Yes,Pass/Fail,None,None,The course wi ll discuss sustainability from p...,Environment Earth Systems
1,HW0001,INTRODUCTION TO ACADEMIC COMMUNICATION,0,Language and Communication Centre,Yes,Pass/Fail,None,None,The course aims to support you in your academi...,Language and Communication
2,HY0001,ETHICS & MORAL REASONING,1,School of Humanities,Yes,Pass/Fail,None,None,HY0001 will introduce students to three major ...,Philosophy
3,LS5005,SPANISH LANGUAGE LEVEL 5,3,Centre for Modern Languages,Yes,Letter Graded,"[[LS9004], [LS5004]]",None,"This blended learning course, based on the fli...",Languages
4,SP0061,SCIENCE & TECHNOLOGY FOR HUMANITY,3,University Scholars Programme,Yes,Letter Graded,None,[CC0007],The course aims to inspire a long-lasting mind...,University Scholars Programme
5,AB0403,DECISION MAKING WITH PROGRAMMING & ANALYTICS,3,Nanyang Business School,Yes,Letter Graded,None,"[AB8401, BC0401]",This is an introductory course designed for bu...,Business/Accountancy
6,AB1201,FINANCIAL MANAGEMENT,3,Nanyang Business School,Yes,Letter Graded,None,"[BU5201, BU8201]","d Income Securities, Derivatives Securities, a...",Business/Accountancy
7,AB1202,STATISTICS & ANALYSIS,3,Nanyang Business School,Yes,Letter Graded,None,"[CE1008, HE1004, MH1820]",This course introduces the concepts and method...,Business/Accountancy
8,AB1301,BUSINESS LAW,3,Nanyang Business School,Yes,Letter Graded,None,[BU8301],The aim of this course is to provide students ...,Business/Accountancy
9,AB1403,INTERMEDIATE EXCEL,1,Nanyang Business School,Yes,Pass/Fail,None,[AB1401],This course focuses on teaching introductory t...,Business/Accountancy
